In [1]:
import sys
import pathlib
import pandas as pd

project_root = pathlib.Path.cwd().resolve()
while not (project_root / "backend").is_dir():
    if project_root.parent == project_root:
        raise RuntimeError("Could not find project root containing 'backend/'")
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from backend.utils import fetch_metrics, country_data_fetch
from backend import constants, data_retrieval, data_push


# Paths
BACKEND_DIR      = project_root / "backend"
PROCESSED_DATA   = BACKEND_DIR / "data" / "wb_panel_wide"
RAW_DATA_EXCEL   = BACKEND_DIR / "data" / "country_data.xlsx"

## Pick One Country

In [2]:
data_path = project_root / "backend" / "data" / "country_data.xlsx"
country_data_df = pd.read_excel(data_path)
countries: list = country_data_df["Country_Name"].values.tolist()

# Testing Program for Random Country
selected_country = countries[-4] # ---- Pick the country here
country_iso_code = country_data_df[country_data_df["Country_Name"] == selected_country]["iso2Code"].values.tolist()[0]

# Fetch All Data Points For A Random Country
country_data = fetch_metrics.build_country_panel(country_iso_code, constants.INDICATORS)
print(f"Country: {selected_country} | Country ISO Code: {country_iso_code}")
country_data

Country: Ukraine | Country ISO Code: UA


,INFLATION,UNEMPLOYMENT,FDI_PCT_GDP,POL_STABILITY,RULE_OF_LAW,CONTROL_CORRUPTION,GINI_INDEX,GDP_PC_GROWTH,INT_PAYM_PCT_REV
1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020,2.732492,9.475,0.194103,-1.163936,-0.713144,-0.832629,25.6,-3.084586,8.769683
2021,9.363139,9.834,3.981661,-1.126984,-0.681738,-0.787212,NaN,4.410509,9.265518
2022,20.183637,NaN,0.136429,-1.949325,-0.917948,-0.634004,NaN,-22.745681,7.258917
2023,12.849022,NaN,2.522879,-1.429823,-0.888317,-0.686146,NaN,15.217865,8.000564


## Create or Update the Country Parquet File

In [3]:
panel = fetch_metrics.build_country_panel(
    country_iso_code,
    indicators=constants.INDICATORS,
    start=None,
    end=None,
    tidy_fetch=True,
)
country_data_fetch.ingest_panel_wide(panel, country_iso_code, root=PROCESSED_DATA,)

## Read Country Parquet File and Prepare Data for LLM Inference

In [4]:
country_context = data_retrieval.prepare_llm_payload_pretty(
    country_iso=country_iso_code,
    indicators=constants.INDICATORS,
    since=2020,
    lookback=5
    )
country_context

{'country': 'UA',
 'latest_year': 2024,
 'indicators': {'Inflation (% y/y)': {'latest': 6.5,
   'Δ1y': -0.494,
   'Δ5y': None,
   'series': {2020: 2.73, 2021: 9.36, 2022: 20.18, 2023: 12.85, 2024: 6.5}},
  'Unemployment (% labour force)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: 9.48, 2021: 9.83}},
  'FDI inflow (% GDP)': {'latest': 1.99,
   'Δ1y': -0.211,
   'Δ5y': None,
   'series': {2020: 0.19, 2021: 3.98, 2022: 0.14, 2023: 2.52, 2024: 1.99}},
  'Political stability (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: -1.16, 2021: -1.13, 2022: -1.95, 2023: -1.43}},
  'Rule of law (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: -0.71, 2021: -0.68, 2022: -0.92, 2023: -0.89}},
  'Control of corruption (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: -0.83, 2021: -0.79, 2022: -0.63, 2023: -0.69}},
  'Income inequality (Gini)': {'latest': None,
   'Δ1y': None,
   'Δ5y'

## Add Country Data and Most Recent Geopolitical/Economic News Links Into LLM Prompt

In [6]:
import json

from backend import constants
from backend import data_retrieval
from backend.ai import langchain_llm
from backend.utils import fetch_links

# ---------------------------------------------------------------------------
# 1. Build the macro-data payload (World Bank panel) -------------------------
# ---------------------------------------------------------------------------

COUNTRY_ISO   = country_iso_code            # Picked Country
RAW_INDICATORS = constants.INDICATORS       # {"INFLATION": "FP.CPI.TOTL.ZG", ...}

payload = data_retrieval.prepare_llm_payload_pretty(
    country_iso = COUNTRY_ISO,
    indicators  = RAW_INDICATORS,
    since       = 2015,
    lookback    = 10,
    deltas      = (1, 5)
)

# Turn the key → pretty-label mapping into the bullet points the prompt expects
prompt_points = ", ".join(payload["indicators"].keys())
# e.g. "Inflation (% y/y), Unemployment (% labour force), …"

# ---------------------------------------------------------------------------
# 2. Grab recent economic / political headlines -----------------------------
# ---------------------------------------------------------------------------

headlines = [
    item["title"]
    for item in fetch_links.gnews_rss(
        country  = selected_country,
        n        = 10,
        days     = 365,
        lang     = "en",
        region   = "US"
    )
]

# ---------------------------------------------------------------------------
# 3. Ask the LLM for a country risk score -----------------------------------
# ---------------------------------------------------------------------------

# result = langchain_llm.country_llm_score(
#     country       = payload["country"],
#     headlines     = headlines,
#     prompt_points = prompt_points
# )

# # ---------------------------------------------------------------------------
# # 4. Combine & persist, ship to front-end, etc. -----------------------------
# # ---------------------------------------------------------------------------

# report = {
#     **payload,                 # macro data
#     "llm_output": result       # {"score": 0.72, "bullet_summary": "..."}
# }

# print(json.dumps(report, indent=2, ensure_ascii=False))
headlines

['OECD Economic Surveys: Ukraine 2025 - OECD',
 'Ukraine’s political infighting gets nasty - The Economist',
 'Russia and Ukraine are locked in an economic war of attrition - Atlantic Council',
 'Topic: Relations with Ukraine - NATO - Homepage',
 'Ukraine’s Presidential Elections Amid War: Political, Legal, and Security Challenges - Wilson Center',
 'Ukraine signs economic deal with U.S. after pressure from Trump for continued aid - PBS',
 'Lessons for rebuilding Ukraine from economic recoveries after natural disasters - CEPR',
 'Fact Sheet: President Donald J. Trump Secures Agreement to Establish United States-Ukraine Reconstruction Investment Fund - U.S. Embassy in Ukraine (.gov)',
 'How Ukraine Remains Resilient, Three Years On - Carnegie Endowment for International Peace',
 'Economic agreement with Ukraine off the table for now, Bessent says - Politico']